# Import

In [27]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivationByType
from mesa.datacollection import DataCollector
import random
from mesa import Agent
from numpy import random, sqrt
from mesa.visualization import CanvasGrid, ModularServer
import numpy as np
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
import matplotlib.pyplot as plt

In [28]:
import solara
from matplotlib.figure import Figure
import random

In [29]:
def get_distance(pos1, pos2):
    return (pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2

# Class cell

In [30]:
class Cell(Agent):
    def __init__(self, unique_id, model, capacities):
        super().__init__(unique_id, model)
        self.capacities = capacities
        self.sugar = capacities[0]
        self.spice = capacities[1]

    def step(self):
        self.regenerate()

    def regenerate(self):
        self.sugar = min(self.sugar + 1, self.capacities[0])
        self.spice = min(self.spice + 1, self.capacities[1])

# Class Trader

In [31]:
class Trader(Agent):
    def __init__(self, unique_id, model, sugar, sugar_metabolism,
                 spice, spice_metabolism, vision, max_age):
        super().__init__(unique_id, model)

        # Set initial parameters
        self.sugar = sugar
        self.sugar_metabolism = sugar_metabolism
        self.spice = spice
        self.spice_metabolism = spice_metabolism
        self.vision = vision
        self.max_age = max_age
        self.age = 0

        # Weight for both sugar and spice when moving
        self.spice_weight = sugar_metabolism / (sugar_metabolism + spice_metabolism)
        self.sugar_weight = 1 - self.spice_weight

        # Set initial wealth
        self.wealth = 0
        self.update_wealth()

        # Initialize trader's price
        self.price = 0

        # If agent has died
        self.has_died = False

    def step(self):
        # Move agent
        self.move()

        # Pick up sugar and spice
        self.pick_up()

        # Update wealth
        self.update_wealth()

        # Trade sugar and spice
        self.trade()

        # Repopulation
        self.repopulate()

        # Metabolize sugar and spice
        self.metabolize()

        # Increment age
        self.age_increase()

        # Check if agent has died
        if self.has_died:
            self.model.remove_agent(self)

    def move(self):
        # Get neighborhood
        neighbours = [i for i in self.model.grid.get_neighborhood(
            self.pos, moore=False, include_center=False, radius=self.vision
        )]

        # Get cell with most sugar
        max_welfare = -1
        best_position = None
        shortest_distance = float('inf')

        for neighbour in neighbours:
            # Check if another trader is in the cell
            this_cell = self.model.grid.get_cell_list_contents([neighbour])

            # Check if trader within cell
            has_agent = False
            for cells in this_cell:
                if isinstance(cells, Trader):
                    has_agent = True
                    break
            if has_agent:
                continue
            agent = this_cell[0]

            # Compute welfare based on sum of current resources and resources in the cell
            combined_sugar = self.sugar + agent.sugar
            combined_spice = self.spice + agent.spice
            welfare = self.welfare(combined_sugar, combined_spice)
            distance = get_distance(self.pos, neighbour)

            # Update best position
            if (welfare > max_welfare) or (welfare == max_welfare and distance < shortest_distance):
                max_welfare = welfare
                best_position = neighbour
                shortest_distance = distance

        if best_position:
            # Move to the position with the highest welfare
            self.model.grid.move_agent(self, best_position)

    def pick_up(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        # Grab all sugar and spice from cell
        for agent in this_cell:
            if isinstance(agent, Cell):
                self.sugar += agent.sugar
                agent.sugar = 0

                self.spice += agent.spice
                agent.spice = 0

    def metabolize(self):
        # Metabolize sugar
        self.sugar -= self.sugar_metabolism

        # Metabolize spice
        self.spice -= self.spice_metabolism

        # Die if sugar is less than 0
        if self.sugar < 0 or self.spice < 0:
            self.model.deaths_starved_step += 1
            self.has_died = True

    def trade(self):
        # Get neighborhood
        neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False, radius=1)
        random.shuffle(neighbors)

        # Loop through neighbors
        for neighbors in neighbors:
            # Skip if not a trader
            if not isinstance(neighbors, Trader):
                continue

            # Allows for continuous trading
            while True:
                # Compute MRS
                mrs = self.mrs()
                neighbors_mrs = neighbors.mrs()

                # No more trading if MRS are equal
                if mrs == neighbors_mrs:
                    break

                # Check who has the higher MRS
                if mrs > neighbors_mrs:
                    high = self
                    low = neighbors
                else:
                    high = neighbors
                    low = self

                # Compute the trade price
                trade_price = sqrt(mrs * neighbors_mrs)

                if trade_price == 0:
                    break

                # Check if trade price is greater than 1
                if trade_price > 1:
                    trade_spice = trade_price
                    trade_sugar = 1
                else:
                    trade_spice = 1
                    trade_sugar = 1 / trade_price

                # Update based on MRS
                trade_sugar = min(trade_sugar, low.sugar)
                trade_spice = min(trade_spice, high.spice)

                # No more sugar/spice to trade
                if trade_sugar <= 0 or trade_spice <= 0:
                    break

                # Check if trade improves welfare
                if self.improve_welfare(high, low, trade_sugar, trade_spice):
                    # Trade sugar and spice
                    high.sugar += trade_sugar
                    high.spice -= trade_spice
                    low.sugar -= trade_sugar
                    low.spice += trade_spice

                    # Update table
                    self.model.datacollector.add_table_row("Trades", {
                        'Step': self.model.current_step,
                        'TraderHighMRS_ID': high.unique_id,
                        'TraderLowMRS_ID': low.unique_id,
                        'TradeSugar': trade_sugar,
                        'TradeSpice': trade_spice,
                        'TradePrice': trade_price
                    })

                # No more improvements
                else:
                    break

    def mrs(self):
        return (self.sugar_metabolism * self.spice) / (self.spice_metabolism * self.sugar + 1e-9)

    def improve_welfare(self, high, low, trade_sugar, trade_spice):
        # Compute welfare
        high_sugar = high.sugar + trade_sugar
        high_spice = high.spice - trade_spice
        low_sugar = low.sugar - trade_sugar
        low_spice = low.spice + trade_spice

        # Compute welfare
        high_welfare = self.welfare(high_sugar, high_spice)
        low_welfare = self.welfare(low_sugar, low_spice)

        # Check if welfare is improved
        improved = high_welfare > high.wealth and low_welfare > low.wealth

        # Make sure that MRS is not crossed
        not_crossed = high_welfare > low_welfare

        # Check if welfare is improved
        return improved and not_crossed

    def repopulate(self):
        # Check if trader has enough sugar and spice
        if (self.sugar >= self.model.repopulate_factor * self.sugar_metabolism
                and self.spice >= self.model.repopulate_factor * self.spice_metabolism):
            # Create new trader
            self.model.repopulation()

            repopulate_loss_ratio = 0.5
            # Reduce sugar and spice
            self.sugar *= 1 - repopulate_loss_ratio
            self.spice *= 1 - repopulate_loss_ratio

    def age_increase(self):
        # Increment age
        self.age += 1

        # Check if age is greater than max age
        if self.age >= self.max_age:
            self.has_died = True
            self.model.deaths_age_step += 1

    def welfare(self, sugar, spice):
        return sugar ** self.sugar_weight * spice ** self.spice_weight

    def update_wealth(self):
        self.wealth = self.welfare(self.sugar, self.spice)

# Class Distributer

## BaseDistributer

In [32]:
class BaseDistributer:
    def __init__(self, distributer_steps) -> None:
        self.distributer_steps = distributer_steps
        self.current_step = 0

    def step(self, agents, taxer):
        self.current_step += 1
        if self.current_step % self.distributer_steps == 0:
            self.distribute(agents, taxer)

    def distribute(self, agents, taxer):
        total_agents = len([agent for agent in agents if isinstance(agent, Trader)])
        if total_agents > 0:
            sugar_per_agent = taxer.taxes_collection["sugar"] / total_agents
            spice_per_agent = taxer.taxes_collection["spice"] / total_agents
            for agent in agents:
                agent.sugar += sugar_per_agent
                agent.spice += spice_per_agent

            # Reset taxes collection
            taxer.reset_tax()


## ProgressiveDistributer

In [33]:
class ProgressiveDistributer(BaseDistributer):
    def distribute(self, agents, taxer):
        # Get wealth of all agents
        wealths = [agent.wealth for agent in agents]
        wealths.sort()

        # Find the threshold for classes
        low_n = len(wealths) // 3 + 1
        middle_n = 2 * low_n
        low_class_threshold = wealths[low_n]
        middle_class_threshold = wealths[middle_n]

        # Find how much each class gets distributed
        low_class = {}
        middle_class = {}
        high_class = {}
        for key in taxer.taxes_collection:
            # Compute how much middle class gets
            middle_class[key] = taxer.taxes_collection[key] / (7 / 3 * low_n + 2 / 3 * (len(wealths) - middle_n))

            # Compute how much low and high class gets
            low_class[key] = middle_class[key] * 4 / 3
            high_class[key] = middle_class[key] * 2 / 3

        # Distribute
        for agent in agents:
            if agent.wealth < low_class_threshold:
                agent.sugar += low_class["sugar"]
                agent.spice += low_class["spice"]

            elif agent.wealth < middle_class_threshold:
                agent.sugar += middle_class["sugar"]
                agent.spice += middle_class["spice"]

            else:
                agent.sugar += high_class["sugar"]
                agent.spice += high_class["spice"]

        # Reset taxes collection
        taxer.reset_tax()

# Class Tax

## BaseTaxer

In [34]:
class BaseTaxer:
    def __init__(self, tax_steps, tax_rate) -> None:
        self.tax_steps = tax_steps
        self.tax_rate = tax_rate
        self.taxes_collection = {"sugar": 0, "spice": 0}
        self.current_step = 0

    def step(self, agents):
        self.current_step += 1
        if self.current_step % self.tax_steps == 0:
            self.collect_taxes(agents)

    def collect_taxes(self, agents):
        for agent in agents:
            # Compute tax
            sugar_tax = agent.sugar * self.tax_rate
            spice_tax = agent.spice * self.tax_rate

            # Update agent's goods and taxes collection
            agent.sugar -= sugar_tax
            agent.spice -= spice_tax
            self.taxes_collection["sugar"] += sugar_tax
            self.taxes_collection["spice"] += spice_tax

    def reset_tax(self):
        self.taxes_collection = {"sugar": 0, "spice": 0}

## ProgressiveTaxer

In [35]:
class ProgressiveTaxer(BaseTaxer):
    def collect_taxes(self, agents):
        # Get wealth distribution to determine tax rates
        wealths = [agent.wealth for agent in agents]

        # Sort wealth
        wealths.sort()

        # Find 33rd and 66th percentiles
        low_class = wealths[len(wealths) // 3]
        middle_class = wealths[2 * len(wealths) // 3]

        # Collect taxes
        for agent in agents:
            if agent.wealth < low_class:
                self.update_goods(agent, self.tax_rate * 0.66)
            elif agent.wealth < middle_class:
                self.update_goods(agent, self.tax_rate)
            else:
                self.update_goods(agent, self.tax_rate * 1.33)

    def update_goods(self, agent, tax_rate):
        self.taxes_collection["sugar"] += int(agent.sugar * tax_rate)
        self.taxes_collection["spice"] += int(agent.spice * tax_rate)
        agent.sugar -= int(agent.sugar * tax_rate)
        agent.spice -= int(agent.spice * tax_rate)

# Class SugarScape

## Global functions

In [36]:
def compute_trade_counts(model):
    trade_data = model.get_trade_log()
    current_step_trades = trade_data[trade_data["Step"] == model.current_step]
    return len(current_step_trades)


def compute_average_trade_price(model):
    trade_data = model.get_trade_log()
    if len(trade_data) == 0:
        return 0
    current_step_trades = trade_data[trade_data["Step"] == model.current_step]
    if len(current_step_trades) == 0:
        return 0
    average_price = current_step_trades["TradePrice"].mean()
    return average_price


def compute_gini(model):
    agent_wealths = [agent.sugar / agent.sugar_metabolism + agent.spice / agent.spice_metabolism for agent in
                     model.traders.values()]
    sorted_wealths = sorted(agent_wealths)
    # plt.hist(sorted_wealths, bins=10)
    # plt.show()
    n = len(sorted_wealths)
    # print(n)
    if n == 0:
        return 0
    cumulative_sum = sum((i + 1) * wealth for i, wealth in enumerate(sorted_wealths))
    total_wealth = sum(sorted_wealths)
    gini = (2 * cumulative_sum) / (n * total_wealth) - (n + 1) / n

    return gini


def compute_deaths_by_age(model):
    """Return the number of deaths by age for the current step."""
    return model.deaths_age[-1] if model.deaths_age else 0


def compute_deaths_by_hunger(model):
    """Return the number of deaths by hunger for the current step."""
    return model.deaths_starved[-1] if model.deaths_starved else 0


def compute_average_vision(model):
    """Compute the average vision of all living Trader agents."""
    traders = [agent for agent in model.traders.values()]
    if len(traders) == 0:
        return 0
    average_vision = sum(trader.vision for trader in traders) / len(traders)
    return average_vision


def compute_average_sugar_metabolism(model):
    """Compute the average sugar metabolism of all living Trader agents."""
    traders = [agent for agent in model.traders.values()]
    if len(traders) == 0:
        return 0
    average_sugar_metabolism = sum(trader.sugar_metabolism for trader in traders) / len(traders)
    return average_sugar_metabolism


def compute_average_spice_metabolism(model):
    """Compute the average spice metabolism of all living Trader agents."""
    traders = [agent for agent in model.traders.values()]
    if len(traders) == 0:
        return 0
    average_spice_metabolism = sum(trader.spice_metabolism for trader in traders) / len(traders)
    return average_spice_metabolism

## Main part

In [37]:
class SugarScape(Model):
    def __init__(self, height=50, width=50, initial_population=100,
                 tax_scheme="flat", tax_steps=10, tax_rate=0.1, distributer_scheme="flat", distributer_steps=20,
                 repopulate_factor=10, seed_value=42):
        super().__init__()
        # Set parameters
        self.height = height
        self.width = width
        self.initial_population = initial_population
        self.current_step = 0
        self.repopulate_factor = repopulate_factor

        # Set seed for reproducibility
        random.seed(seed_value)

        self.schedule = RandomActivationByType(self)
        self.grid = MultiGrid(self.height, self.width, False)

        # Initialize counters
        self.deaths_age = []
        self.deaths_starved = []
        self.deaths_age_step = 0
        self.deaths_starved_step = 0
        self.reproduced = 0

        # Create taxers and distributers
        if tax_scheme == "flat":
            self.taxer = BaseTaxer(tax_steps, tax_rate)
        elif tax_scheme == "progressive":
            self.taxer = ProgressiveTaxer(tax_steps, tax_rate)
        else:
            raise ValueError("Invalid tax scheme")

        if distributer_scheme == "flat":
            self.distributer = BaseDistributer(distributer_steps)
        elif distributer_scheme == "progressive":
            self.distributer = ProgressiveDistributer(distributer_steps)
        else:
            raise ValueError("Invalid distributer scheme")

        # Create cells
        id = 0
        for content, (x, y) in self.grid.coord_iter():
            # Define capacities and reproduction rates based on location
            if x < self.width // 2 and y < self.height // 2:  # Left Upper
                capacities = [random.randint(5, 10), random.randint(0, 2)]
            elif x < self.width // 2 and y >= self.height // 2:  # Left Lower
                capacities = [random.randint(5, 10), random.randint(0, 2)]
            elif x >= self.width // 2 and y < self.height // 2:  # Right Upper
                capacities = [random.randint(0, 2), random.randint(5, 10)]
            else:  # Right Lower
                capacities = [random.randint(0, 2), random.randint(5, 10)]

            cell = Cell(id, self, capacities)

            # Place cell on grid
            self.grid.place_agent(cell, (x, y))
            self.schedule.add(cell)

            # Increment id
            id += 1

        # Create traders
        self.traders = {}
        for i in range(self.initial_population):
            # Random position
            x = random.randint(0, self.width - 1)
            y = random.randint(0, self.height - 1)

            # Instantiate trader
            sugar, spice = np.random.randint(1, 10, 2)
            sugar_metabolism, spice_metabolism = np.random.randint(2, 8, 2)
            vision = random.randint(1, 4)
            max_age = random.randint(70, 100)
            trader = Trader(id, self, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age)

            # Place trader on grid
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)

            # Add trader to dictionary
            self.traders[id] = trader

            # Increment id
            id += 1

        self.datacollector = DataCollector(
            model_reporters={
                "Trade Price": compute_average_trade_price,
                "Gini": compute_gini,
                "Number of Trades": compute_trade_counts,
                "Deaths by Age": compute_deaths_by_age,
                "Deaths by Hunger": compute_deaths_by_hunger,
                "Average Vision": compute_average_vision,
                "Average Sugar Metabolism": compute_average_sugar_metabolism,
                "Average Spice Metabolism": compute_average_spice_metabolism,
                "Reproduced": lambda m: m.reproduced
            },
            tables={"Trades": ["Step", "TraderHighMRS_ID", "TraderLowMRS_ID", "TradeSugar", "TradeSpice", "TradePrice"]}
        )

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.current_step += 1
        self.deaths_age_step = 0
        self.deaths_starved_step = 0
        self.schedule.step()

        self.deaths_age.append(self.deaths_age_step)
        self.deaths_starved.append(self.deaths_starved_step)

        # Get all trader
        traders = [agent for agent in self.schedule.agents if isinstance(agent, Trader)]

        # Take step for taxer and distributer
        self.taxer.step(traders)
        self.distributer.step(traders, self.taxer)

        self.datacollector.collect(self)
        self.running = True if self.schedule.get_agent_count() > 0 else False

    def run_model(self, step_count=200):
        for i in range(step_count):
            self.step()

    def get_trade_log(self):
        return self.datacollector.get_table_dataframe("Trades")

    def remove_agent(self, agent):
        self.grid.remove_agent(agent)
        self.schedule.remove(agent)
        del self.traders[agent.unique_id]

    def repopulation(self):
        # Random position
        x = random.randint(0, self.width - 1)
        y = random.randint(0, self.height - 1)

        # Instantiate trader
        sugar, spice = np.random.randint(1, 10, 2)
        sugar_metabolism, spice_metabolism = np.random.randint(2, 8, 2)
        vision = random.randint(1, 4)
        max_age = random.randint(70, 100)
        id = max(self.traders.keys()) + 1
        trader = Trader(id, self, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age)

        # Place trader on grid
        self.grid.place_agent(trader, (x, y))
        self.schedule.add(trader)

        # Add trader to dictionary
        self.traders[id] = trader
        """
        # Get distribution of metabolism, vision and max age
        sugar_metabolism = {i: 0 for i in range(1, 4)}
        spice_metabolism = {i: 0 for i in range(1, 4)}
        vision = {i: 0 for i in range(1, 4)}
        max_age = {i: 0 for i in range(70, 100)}

        # Incrementing each level within distribution
        for trader in self.traders.values():
            sugar_metabolism[trader.sugar_metabolism] += 1
            spice_metabolism[trader.spice_metabolism] += 1
            vision[trader.vision] += 1
            max_age[trader.max_age] += 1

        # Normalize distribution
        n = len(self.traders)
        sugar_metabolism = {k: v/n for k, v in sugar_metabolism.items()}
        spice_metabolism = {k: v/n for k, v in spice_metabolism.items()}
        vision = {k: v/n for k, v in vision.items()}
        max_age = {k: v/n for k, v in max_age.items()}

        # Use distributions to create set of parameters
        sugar_metabolism = random.choice(list(sugar_metabolism.keys()), p=list(sugar_metabolism.values()))
        spice_metabolism = random.choice(list(spice_metabolism.keys()), p=list(spice_metabolism.values()))
        vision = random.choice(list(vision.keys()), p=list(vision.values()))
        max_age = random.choice(list(max_age.keys()), p=list(max_age.values()))

        # Create new trader
        id = max(self.traders.keys()) + 1
        sugar, spice = np.random.randint(1, 10, 2)
        trader = Trader(id, self, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age)

        # Random position
        x = random.randint(0, self.width)
        y = random.randint(0, self.height)

        # Place trader on grid
        self.grid.place_agent(trader, (x, y))
        self.schedule.add(trader)

        # Add trader to dictionary
        self.traders[id] = trader
        """
        # Increment reproduction counter
        self.reproduced += 1


# Run model

In [38]:
SugarScape().run_model()

In [39]:
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {"Filled": "true",
                 "r": 0.5,
                 "w": 1,
                 "h": 1}

    if type(agent) is Trader:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 1
        portrayal["Shape"] = "circle"
    elif type(agent) is Cell:
        portrayal["Shape"] = "rect"
        portrayal["Color"] = "green" if agent.sugar > 0 and agent.spice > 0 else "black"
        portrayal["Layer"] = 0

    return portrayal


canvas_element = CanvasGrid(agent_portrayal, 50, 50, 500, 500)

trade_count_chart = ChartModule(
    [{"Label": "Number of Trades", "Color": "Blue"}],
    data_collector_name='datacollector'
)

average_trade_price_chart = ChartModule(
    [{"Label": "Trade Price", "Color": "Red"}],
    data_collector_name='datacollector'
)

gini_pop = ChartModule(
    [{"Label": "Gini", "Color": "Black"}],
    data_collector_name='datacollector'
)

deaths_by_age_chart = ChartModule(
    [{"Label": "Deaths by Age", "Color": "Green"}],
    data_collector_name='datacollector'
)

deaths_by_hunger_chart = ChartModule(
    [{"Label": "Deaths by Hunger", "Color": "Orange"}],
    data_collector_name='datacollector'
)

average_vision_chart = ChartModule(
    [{"Label": "Average Vision", "Color": "Purple"}],
    data_collector_name='datacollector'
)

average_sugar_metabolism_chart = ChartModule(
    [{"Label": "Average Sugar Metabolism", "Color": "Pink"}],
    data_collector_name='datacollector'
)

average_spice_metabolism_chart = ChartModule(
    [{"Label": "Average Spice Metabolism", "Color": "Brown"}],
    data_collector_name='datacollector'
)

reproduced_chart = ChartModule(
    [{"Label": "Reproduced", "Color": "Black"}],
    data_collector_name='datacollector'
)

server = ModularServer(
    SugarScape,
    [canvas_element, trade_count_chart, average_trade_price_chart, gini_pop,
     deaths_by_age_chart, deaths_by_hunger_chart, average_vision_chart,
     average_sugar_metabolism_chart, average_spice_metabolism_chart, reproduced_chart],
    "Sugarscape Model",
    {"height": 50, "width": 50, "initial_population": 100}
)

server.port = 8563
server.launch()

Interface starting at http://127.0.0.1:8563


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step